In [6]:
# Initial imports
import os # Miscellaneous operating system interfaces (os.getenv)
import requests # requests.get
import pandas as pd
from dotenv import load_dotenv #Reads the key-value pair from .env file and adds them to environment variable.
from pathlib import Path
import json # lightweight data interchange format 
import matplotlib.pyplot as plt

%matplotlib inline
import FundamentalAnalysis as fa
import alpaca_trade_api as tradeapi
import FundamentalAnalysis as fa
import alpaca_trade_api as tradeapi
import panel as pn
import plotly.express as px
import hvplot.pandas
import plotly.graph_objects as go
import numpy as np
from panel.interact import interact
from panel import widgets

%matplotlib inline

ModuleNotFoundError: No module named 'dotenv'

In [5]:
# Load .env enviroment variables
load_dotenv()
from pathlib import Path
env_path = Path('C:/Users/Nipune/python-homework/.env')

NameError: name 'load_dotenv' is not defined

In [3]:
# Set Fundamental Analysis API key
fa_api_key = os.getenv("FINANCIAL_MODELING_PREP_API_KEY")

In [4]:
# Confirm API correctly set
type(fa_api_key)

NoneType

In [5]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

NameError: name 'tradeapi' is not defined

In [6]:
# Confirm API correctly set
type(alpaca_api_key)

NoneType

In [ ]:
# Set the file path for S&P 500 data sources
sp_data = Path("Resources/sp.csv")
sp_symbols_data = Path("Resources/sp500_symbols.csv")
sp_ticker_data = Path("Resources/sp500_ticker.csv")

In [ ]:
# Read SP 500 data CSV files and set the `date` column as a datetime index to the DataFrame
sp_df = pd.read_csv(sp_data, index_col="Date", infer_datetime_format=True, parse_dates=True)
sp_df.sort_index(ascending = True, inplace = True)

sp_symbols = pd.read_csv(sp_symbols_data, index_col="Symbol", infer_datetime_format=True, parse_dates=True)
sp_symbols.sort_index(ascending = True, inplace = True)

sp_ticker = pd.read_csv(sp_ticker_data, index_col="Symbol", infer_datetime_format=True, parse_dates=True)
sp_ticker.sort_index(ascending = True, inplace = True)

sp_df.head()

In [ ]:
# Drop nulls and confirm
sp_df = sp_df.dropna().copy()

sp_df.isnull().sum()

In [ ]:
# Drop duplicates and confirm
sp_df = sp_df.drop_duplicates().copy()

sp_df.duplicated().sum()

In [ ]:
# Add the "SP 500" column index and check
index_columns = [("SP 500", "open"), ("SP 500", "high"), ("SP 500", "low"), ("SP 500", "close"), ("SP 500", "volume")]
sp_df.columns = pd.MultiIndex.from_tuples(index_columns)
sp_df.head()

In [ ]:
# Remove time stamp from DateIndex and check
sp = sp_df.reset_index()
sp["Date"] = sp["Date"].astype(str).str[:-8]
sp_500_data_df = sp.set_index("Date")

In [ ]:
sp_500_data_df.head()

In [ ]:
# Calculate daily percentage change for SP 500 stocks
sp_500_daily_change_df = sp_500_data_df.pct_change()
sp_500_daily_change_df.head()

In [ ]:
# Get top 10 foreign stocks purchased by Aussies from Jan 2019 to Dec 2020

# Format current date as ISO format
today = pd.Timestamp("2020-12-24", tz="America/New_York").isoformat() 
start_date = pd.Timestamp("2019-01-01", tz="America/New_York").isoformat()   
end_date = pd.Timestamp("2020-12-31", tz="America/New_York").isoformat()   

# Set the tickers for most foreign stocks purchased by Aussies
tickers = ["TSLA", "AAPL", "PFE", "MSFT", "MRNA", "ZM", "AMZN", "NNDM", "CRM", "NVDA"]

#Set variables
dict = {'stock':'', 'segment':''}  
marketcaps = {}
segments = {}
for stock in tickers:
    profile = fa.profile(stock, fa_api_key) 
    try: 
        marketcaps[stock] = int(profile.iloc[22])
        segment = str(profile.iloc[26])[5:]
        segments[stock] = segment[:-28]
    except: continue
    
# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get 2-year daily data for most foreign stocks purchased by Aussies
foreign_stocks_df = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

In [ ]:
profile

In [ ]:
# Adjust date and time format to match SP 500 data. Remove timezome and time stamp

top_10 = foreign_stocks_df.reset_index().rename(columns={"time" : "Date"}) 
top_10["Date"] = top_10["Date"].astype(str).str[:-14]
foreign_stock_data_df = top_10.set_index("Date")

foreign_stock_data_df.head()

In [ ]:
# Calculate daily percentage change for most foreign stocks purchased by Aussies
foreign_stocks_daily_change_df = foreign_stock_data_df.pct_change()
foreign_stocks_daily_change_df.head()

In [ ]:
# Join 2-Year SP 500 data returns to that of foreign stock mostly purchased by Aussies
combined_data_daily_change_df = pd.concat([foreign_stocks_daily_change_df, sp_500_daily_change_df], axis = "columns", join = "inner")
combined_data_daily_change_df.tail()

In [ ]:
# Determine market capitalization and segments of top 10 foreign stocks purchased by Aussies
marketcaps_df = pd.DataFrame(marketcaps.items(), columns=['stock', 'marketcap'])
segments_df = pd.DataFrame(segments.items(), columns=['stock', 'marketcap']).rename(columns={
    "marketcap":"sector"
})

In [ ]:
marketcaps_df

In [ ]:
segments_df

In [ ]:
# Drop columns
segments = segments_df.drop(columns=[
    "stock", 
])

In [ ]:
# Create dataframe showing market capitalization and sectors for top 10 foreign stocks purchased by Aussies
joined_df = pd.concat([marketcaps_df, segments], axis="columns", join="inner")
joined_df.head(12)


In [ ]:
# Create dataframe showing decription of SP 500 stocks  
joined_symbol_df = pd.concat([sp_symbols, sp_ticker], axis="columns", join="inner")
joined_symbol_df.head(500)

In [ ]:
key_metrics_quarterly = fa.key_metrics("AAPL", fa_api_key, period="quarter")

In [ ]:
key_metrics_quarterly

In [ ]:
#stock_data = fa.stock_data("AAPL", period="ytd", interval="1d")

In [ ]:
#growth_annually = fa.financial_statement_growth("TSLA", fa_api_key, period="annual")

In [ ]:
#ratings = fa.rating("MSFT", fa_api_key)

In [ ]:
#stock_data_detailed = fa.stock_data_detailed("MSFT", fa_api_key, begin="2000-01-01", end="2001-01-01")

In [ ]:
# # Collect general company information
# profile = fa.profile(ticker, api_key)

#profile = fa.profile(stock, api_key) 

# # Collect recent company quotes
# quotes = fa.quote(ticker, api_key)

# # Collect market cap and enterprise value
# entreprise_value = fa.enterprise(ticker, api_key)

# # Show recommendations of Analysts
# ratings = fa.rating(ticker, api_key)

# # Obtain DCFs over time
# dcf_annually = fa.discounted_cash_flow(ticker, api_key, period="annual")
#dcf_quarterly = fa.discounted_cash_flow(ticker, api_key, period="quarter")

# # Collect the Balance Sheet statements
# balance_sheet_annually = fa.balance_sheet_statement(ticker, api_key, period="annual")
# balance_sheet_quarterly = fa.balance_sheet_statement(ticker, api_key, period="quarter")

# # Collect the Income Statements
# income_statement_annually = fa.income_statement(ticker, api_key, period="annual")
# income_statement_quarterly = fa.income_statement(ticker, api_key, period="quarter")

# # Collect the Cash Flow Statements
# cash_flow_statement_annually = fa.cash_flow_statement(ticker, api_key, period="annual")
# cash_flow_statement_quarterly = fa.cash_flow_statement(ticker, api_key, period="quarter")

# # Show Key Metrics
# key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
# key_metrics_quarterly = fa.key_metrics(ticker, api_key, period="quarter")

# # Show a large set of in-depth ratios
# financial_ratios_annually = fa.financial_ratios(ticker, api_key, period="annual")
# financial_ratios_quarterly = fa.financial_ratios(ticker, api_key, period="quarter")

# # Show the growth of the company
# growth_annually = fa.financial_statement_growth(ticker, api_key, period="annual")
# growth_quarterly = fa.financial_statement_growth(ticker, api_key, period="quarter")

# # Download general stock data
# stock_data = fa.stock_data(ticker, period="ytd", interval="1d")

# # Download detailed stock data
# stock_data_detailed = fa.stock_data_detailed(ticker, api_key, begin="2000-01-01", end="2020-01-01")


In [ ]:
# Show the available companies
#companies = fa.available_companies(api_key)

# Collect general company information
#profile = fa.profile(ticker, api_key)

# Collect recent company quotes
#quotes = fa.quote(ticker, api_key)

# Collect market cap and enterprise value
#entreprise_value = fa.enterprise(ticker, api_key)

# Show recommendations of Analysts
#ratings = fa.rating(ticker, api_key)

# Obtain DCFs over time
#dcf_annually = fa.discounted_cash_flow(ticker, api_key, period="annual")
#dcf_quarterly = fa.discounted_cash_flow(ticker, api_key, period="quarter")

# Collect the Balance Sheet statements
#balance_sheet_annually = fa.balance_sheet_statement(ticker, api_key, period="annual")
#balance_sheet_quarterly = fa.balance_sheet_statement(ticker, api_key, period="quarter")

# Collect the Income Statements
#income_statement_annually = fa.income_statement(ticker, api_key, period="annual")
#income_statement_quarterly = fa.income_statement(ticker, api_key, period="quarter")

# Collect the Cash Flow Statements
#cash_flow_statement_annually = fa.cash_flow_statement(ticker, api_key, period="annual")
#cash_flow_statement_quarterly = fa.cash_flow_statement(ticker, api_key, period="quarter")

# Show Key Metrics
#key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
###key_metrics_quarterly = fa.key_metrics(ticker, api_key, period="quarter")

# Show a large set of in-depth ratios
#financial_ratios_annually = fa.financial_ratios(ticker, api_key, period="annual")
#financial_ratios_quarterly = fa.financial_ratios(ticker, api_key, period="quarter")

# Show the growth of the company
#growth_annually = fa.financial_statement_growth(ticker, api_key, period="annual")
#growth_quarterly = fa.financial_statement_growth(ticker, api_key, period="quarter")

# Download general stock data
#stock_data = fa.stock_data(ticker, period="ytd", interval="1d")

# Download detailed stock data
#stock_data_detailed = fa.stock_data_detailed(ticker, api_key, begin="2000-01-01", end="2020-01-01")